In [460]:
import numpy as np
import xml.etree.ElementTree as ET
import plotly.express as px
import pandas as pd
from typing import List, Dict

# tree = ET.parse("../recordings/api.xml")
tree = ET.parse("/home/yiguo/Desktop/BiliRec_tmp/tmp/47867-20220629-094949-怪物猎人 曙光.xml")
root = tree.getroot()


In [461]:
info = {}
danmaku = []
for child in root:
    if child.tag == "BililiveRecorderRecordInfo":
        info = child.attrib
        print(child.text)
    elif child.tag == "d":
        danmaku.append(
            {
                "time": child.attrib["p"].split(",")[0],
                "user": child.attrib["user"],
                "text": child.text,
            }
        )

for key in info:
    print(key + ": " + info[key])
for d in danmaku[:5]:
    print(d)
print("...")


None
roomid: 47867
shortid: 213
name: C酱です
title: 怪物猎人 曙光
areanameparent: 单机游戏
areanamechild: 主机游戏
start_time: 2022-06-29T09:49:49.4303978-07:00
{'time': '5.148', 'user': '起来你压到我胸毛了', 'text': '这只雪鬼兽是不是要大点'}
{'time': '6.154', 'user': 'AMX-CDC', 'text': '哈哈哈哈哈哈'}
{'time': '6.154', 'user': '213情报局', 'text': '移动端双击屏幕3次，就可获得亲密度哦！'}
{'time': '7.181', 'user': 'Absolute-0', 'text': '血压血压'}
{'time': '7.181', 'user': '老汤面Cheemz', 'text': 'hhhhh'}
...


In [462]:
danmaku_whitelist = {
    "？": [
        "？？",
        "??",
        "问号"],
    "乐":[
        "哈哈",
        "233","笑死"],
    "草": ["草", "艹", "cao"],
    "甜": ["甜","kdl", "磕", "ttk"],
    "xx": ["xx"],
    "酷": ["酷"],
    "三不离": ["三不离", "三句不离"],
    "不好说": ["不好说", "暗恋"],
    "典":["血口喷人", "我会那么笨吗","内心的想法","攻击性"],
    "两眼":["两眼","眼前","双眼","双目"],
    "逆天":["逆天","变态"],
    "汗":["汗"],
    "可爱": ["可爱"],
    "好": ["好好","好耶","棒"],
    "确实": ["确实","正确","合理"],
    "小肉包": ["小肉包", "肉包", "同事", "C酱", "c酱", "双马尾", "闪闪发光", "非常漂亮"],
    "api": ["pi", "Pi", "PI"],
}

danmaku_whitelist_exact = {
    "？":["？", "?"],
    "乐":["乐"],
    "好": ["好"],
}

danmaku_emphasis = ["？", "甜", "三不离", "不好说", "可爱", "两眼"]
emphasis_factor = 2.0

In [463]:
p = {}
time_list_tot = []
for d in danmaku:
    text = d["text"]
    time = round(float(d["time"]))
    weight = 1.0
    # filter and categorize danmaku
    valid = False
    for key in danmaku_whitelist_exact:
        if text in danmaku_whitelist_exact[key]:
            text = key
            valid = True
            break
    for key in danmaku_whitelist:
        if not valid:
            for s in danmaku_whitelist[key]:
                if s in text:
                    text = key
                    valid = True
                    break
        else:
            break
    if not valid:
        continue
    # apply emphasis factor
    if text in danmaku_emphasis:
        weight *= emphasis_factor
    # add danmaku to pool
    item = {"time": time, "weight": weight}
    if text in p:
        p[text].append(item)
    else:
        p[text] = [item]
    # add time to total time list
    time_list_tot.append(item)

time_list_text = p
time_list_text_sorted = [{"text": text, "time_list": p[text]} for text in p]


def sort_key(e: Dict[str, any]):
    return len(e["time_list"])


time_list_text_sorted.sort(reverse=True, key=sort_key)

for item in time_list_text_sorted:
    print(item["text"], len(item["time_list"]))



乐 234
？ 123
好 77
小肉包 23
确实 21
可爱 15
两眼 4
不好说 2
api 2
酷 2
逆天 1
甜 1
xx 1


In [464]:
def calc_freq_map(time_list: List[int], max_time: int):
    average_duration = 15
    map_step_size = 5

    # the input list should have been already in ascending order, but we make sure about it again
    sort_key = lambda e: e["time"]
    time_list.sort(key=sort_key)
    # calculate the time averaged frequency
    x = [_ for _ in range(0, max_time, map_step_size)]
    y = [0 for _ in range(0, max_time, map_step_size)]
    hi, lo = 0, 0
    max_index = len(time_list) - 1
    for i in range(len(x)):
        left = x[i] - average_duration / 2
        right = x[i] + average_duration / 2
        while time_list[lo]["time"] < left and lo < max_index:
            lo += 1
        while time_list[hi]["time"] < right and hi < max_index:
            hi += 1
        tot = 0
        for j in range(lo, hi):
            tot += time_list[j]["weight"]
        y[i] = tot / average_duration

    return x, y


In [465]:
max_time = 0
tot = 0
for item in time_list_tot:
    if item["time"] > max_time:
        max_time = item["time"]
    tot += item["weight"]
average_frequency = tot / max_time

print("max time:", max_time)
print("average frequency:", average_frequency)

max time: 3298
average frequency: 0.19739235900545785


In [466]:
threshold = max(3 * average_frequency, 2)
backward_time = 60
afterward_time = 30

data = {}
for key in time_list_text:
    x, y = calc_freq_map(time_list_text[key], max_time)
    data[key] = y

hotspot_list = []
x, y = calc_freq_map(time_list_tot, max_time)
for i in range(1, len(x) - 1):
    if y[i] > threshold and y[i] >= y[i - 1] and y[i] >= y[i + 1]:
        start = x[i] - backward_time
        end = x[i] + afterward_time
        hotspot = {"time": x[i], "start": start if start > 0 else 0, "end": end if end < max_time else max_time}
        if len(hotspot_list) > 0 and hotspot_list[-1]["end"] > hotspot["start"]:
            hotspot_list[-1]["end"] = hotspot["end"]
        else:
            hotspot_list.append(hotspot)

tot = 0
for item in hotspot_list:
    tot += item["end"] - item["start"]
print("Number of clips:", len(hotspot_list))
print("Clip total length:", tot)
print("Compress ratio:", tot / max_time)


Number of clips: 2
Clip total length: 185
Compress ratio: 0.05609460278956944


In [467]:
selected_danmaku = [item["text"] for item in time_list_text_sorted]

time_list_selected = {"All": time_list_tot}
for text in selected_danmaku:
    time_list_selected[text] = time_list_text[text]


data = []
data.append(["threshold", 0, threshold])
data.append(["threshold", max_time, threshold])
for text in time_list_selected:
    x, y = calc_freq_map(time_list_selected[text], max_time)
    for i in range(len(x)):
        data.append([text, x[i], y[i]])
for item in hotspot_list:
    data.append([str(item["time"]), item["start"], threshold + 1])
    data.append([str(item["time"]), item["end"], threshold + 1])
df = pd.DataFrame(data, columns=["Danmaku", "Time", "Frequency"])

fig = px.line(df, x="Time", y="Frequency", color="Danmaku")
fig.show()


In [468]:
data = []
x, y = calc_freq_map(time_list_tot, max_time)
for i in range(len(x)):
    data.append([text, x[i], y[i]])
df = pd.DataFrame(data, columns=["Danmaku", "Time", "Frequency"])

fig = px.histogram(df, x="Frequency")
fig.show()